In [1]:
import os
import wget
import cv2
from utilis import CUSTOM_MODEL_NAME, PRETRAINED_MODEL_NAME, PRETRAINED_MODEL_URL, TF_RECORD_SCRIPT_NAME
from utilis import LABEL_MAP_NAME, paths, files, labels
import cv2 as cv
import time
import uuid

### 0. Get images

In [2]:
cap = cv.VideoCapture(0)
openPath = os.path.join('images', 'click')
start = time.time()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    img = cv.flip(img, 1)
    cv.imshow('frame', img)
    if cv.waitKey(1) == ord('q'):
        break

    now = time.time()
    if now - start > 3.:
        print("Picture saved!")
        cv.imwrite(os.path.join(openPath, f"{str(uuid.uuid4())[:7]}.jpg"), img)
        start = time.time()

cap.release()
cv.destroyAllWindows()

Picture saved!


In [2]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [4]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https: // github.com / tensorflow / models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 75693, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 75693 (delta 86), reused 108 (delta 56), pack-reused 75538
Receiving objects: 100% (75693/75693), 580.72 MiB | 20.43 MiB/s, done.
Resolving deltas: 100% (53725/53725), done.


### Install Tensorflow Object Detection 

In [5]:
if os.name == 'posix':
    !apt-get install protobuf-compiler
    !cd Tensorflow / models / research & & protoc object_detection / protos / *.proto --python_out=.& & cp object_detection / packages / tf2 / setup.py.& & python -m pip install.

if os.name == 'nt':
    url = "https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} & & tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))
    !cd Tensorflow / models / research & & protoc object_detection / protos / *.proto --python_out=.& & copy object_detection\\packages\\tf2\\setup.py setup.py & & python setup.py build & & python setup.py install
    !cd Tensorflow / models / research / slim & & pip install -e.

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
Processing /media/panjacob/ssd2/Magisterka/Tensorflow/models/research
  Preparing metadata (setup.py) ... done
  Using cached avro_python3-1.10.2-py3-none-any.whl
  Using cached apache_beam-2.40.0-py3-none-any.whl
  Using cached tf_slim-1.1.0-py2.py3-none-any.whl (352 kB)
  Using cached pycocotools-2.0.4-cp310-cp310-linux_x86_64.whl
  Using cached lvis-0.5.3-py3-none-any.whl (14 kB)
  Using cached pandas-1.4.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached tf_models_official-2.9.2-py2.py3-none-any.whl (2.1 MB)
  Using cached tensorflow_io-0.26.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (25.9 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached tensorflow_datasets-4.6.0-py3-none-any.whl (4.3 MB)
  Using cached kaggle-1.5.12-py3-none-a

In [6]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders',
                                   'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2022-07-29 20:00:08.486993: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/panjacob/ssd2/Magisterka/venv/lib/python3.10/site-packages/cv2/../../lib64:
2022-07-29 20:00:08.487013: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-29 20:00:10.349173: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/panjacob/ssd2/Magisterka/venv/lib/python3.10/site-packages/cv2/../../lib64:
2022-07-29 20:00:10.349192: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-29 20:00:10.349204: I tensorflow/stream_executor/cuda/cud

In [3]:
import object_detection

### Import pretrained model

In [4]:
if os.name == 'posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} & & tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} & & tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2022-07-29 20:14:38--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.203.208, 2a00:1450:401b:803::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.203.208|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  51.8MB/s    in 0.4s    

2022-07-29 20:14:39 (51.8 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

/bin/bash: -c: line 1: syntax error near unexpected token `&'
/bin/bash: -c: line 1: `cd Tensorflow/workspace/pre-trained-models & & tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'


# 2. Create Label Map

In [2]:
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records

In [3]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https: // github.com / nicknochnack / GenerateTFRecord {paths['SCRIPTS_PATH']}

In [4]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')}
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 4. Copy Model Config to Training Folder

In [16]:
if os.name == 'posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

cp: cannot stat 'Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config': No such file or directory


# 5. Update Config For Transfer Learning

In [17]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

2022-07-29 20:25:38.474116: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/panjacob/ssd2/Magisterka/venv/lib/python3.10/site-packages/cv2/../../lib64:
2022-07-29 20:25:38.474135: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [18]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

NotFoundError: Tensorflow/workspace/models/my_ssd_mobnet_15000/pipeline.config; No such file or directory

In [ ]:
# config

In [73]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [74]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
# pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [
    os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
# pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [
    os.path.join(paths['ANNOTATION_PATH'], 'train.record')]


In [75]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

# 6. Train the model

In [5]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [6]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=4000".format(TRAINING_SCRIPT,
                                                                                             paths['CHECKPOINT_PATH'],
                                                                                             files['PIPELINE_CONFIG'])

In [7]:
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet_15000 --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet_15000/pipeline.config --num_train_steps=4000


In [109]:
!tensorboard --logdir=./ Tensorflow / workspace / models / my_ssd_mobnet / train

^C


In [29]:
!{command}
# pip install -U tf-models-official --user

^C


# 7. Evaluate the Model

In [27]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT,
                                                                                          paths['CHECKPOINT_PATH'],
                                                                                          files['PIPELINE_CONFIG'],
                                                                                          paths['CHECKPOINT_PATH'])

In [28]:
print(command)

python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\my_ssd_mobnet_8000 --pipeline_config_path=Tensorflow\workspace\models\my_ssd_mobnet_8000\pipeline.config --checkpoint_dir=Tensorflow\workspace\models\my_ssd_mobnet_8000


In [34]:
!{command}

^C
